In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import math
import re
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE,ADASYN
from collections import Counter 
from sklearn.metrics import f1_score 
from imblearn.under_sampling import ClusterCentroids

In [2]:
path = "lab3-data.csv"
data = pd.read_csv(path,encoding = "utf-8",low_memory=False)

## KNN模型预测

- 基于过采样的KNN

In [5]:
def KNN_1(chosen_list) :
    #chosen_list是自变量特征
    chosen_list.append("REPEAT")
    df=data[chosen_list]   #取出所对应的列
    df=df.fillna(method='ffill')   
    df=df.fillna(method='bfill')   #数据预处理采用前后值填充方法
    kf = StratifiedKFold(n_splits = 5,random_state=2023,shuffle=True)    #分层kfold
    scaler = StandardScaler() #数据标准化
    X=np.array(df[chosen_list[:-1]]) #X为自变量特征
    y=np.array(df["REPEAT"])   #y是因变量特征（需要预测的特征）
    oversample = RandomOverSampler(sampling_strategy=1)   #过采样
    f1_score_aver=0.0
    acc_aver=0.0
    acc_aver=0.0
    for train,valid in kf.split(X,y) :
        #split(X,y)将训练集取4份，预测集取1份，train,vaild返回索引两者索引，轮换取
        X_train=np.array(X)[train]   #X_train是训练集自变量特征对应的数据
        y_train=np.array(y)[train]   #y_train是训练集因变量特征对应的数据
        X_test =np.array(X)[valid]   #X_test是预测集自变量特征对应的数据
        y_test=np.array(y)[valid]    #y_test是预测集因变量特征对应的数据
        scaler.fit(X_train)   
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        #根据X_train将X_test与X_train归一化处理，可以收敛更快
        X_train_over, y_train_over = oversample.fit_resample(X_train, y_train)   #由于数据集分布不均匀，所以采用过采样方式
        clf = KNeighborsClassifier(n_neighbors=2)   
        clf.fit(X_train_over, np.array(y_train_over).ravel())  #利用过采样后的训练集训练KNN分类器
        y_pred = clf.predict(X_test)   #将预测集的自变量特征对应的数据X_test输入分类器预测方法得到y_pred预测
        print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))   #acc
        print('F1-score: %.2f' % f1_score(y_test, y_pred))   #f1_score
        f1_score_aver+=f1_score(y_test, y_pred)
        acc_aver+=accuracy_score(y_test, y_pred)
    return [acc_aver/5,f1_score_aver/5]

In [6]:
KNN_1(['OCOD1','OCOD2','OCOD3'])

Accuracy: 0.73
F1-score: 0.29
Accuracy: 0.73
F1-score: 0.29
Accuracy: 0.72
F1-score: 0.30
Accuracy: 0.72
F1-score: 0.28
Accuracy: 0.73
F1-score: 0.29


[0.7277359963776212, 0.2910235469527633]

In [14]:
KNN_1(['COBN_S' ,'COBN_M' ,'COBN_F'])

Accuracy: 0.78
F1-score: 0.08
Accuracy: 0.76
F1-score: 0.23
Accuracy: 0.78
F1-score: 0.08
Accuracy: 0.77
F1-score: 0.14
Accuracy: 0.78
F1-score: 0.06


[0.7738285632214438, 0.11924081330067393]

In [7]:
KNN_1(["PROGN"])

Accuracy: 0.80
F1-score: 0.09
Accuracy: 0.80
F1-score: 0.08
Accuracy: 0.80
F1-score: 0.16
Accuracy: 0.81
F1-score: 0.24
Accuracy: 0.80
F1-score: 0.13


[0.8047705615951957, 0.14096430600613263]

In [8]:
KNN_1(['ST023Q01TA','ST023Q02TA','ST023Q03TA','ST023Q04TA','ST023Q05TA'])

Accuracy: 0.79
F1-score: 0.03
Accuracy: 0.79
F1-score: 0.03
Accuracy: 0.79
F1-score: 0.04
Accuracy: 0.79
F1-score: 0.03
Accuracy: 0.79
F1-score: 0.05


[0.7896671026000572, 0.034546071826525485]

In [9]:
KNN_1(['SCCHANGE'])

Accuracy: 0.79
F1-score: 0.00
Accuracy: 0.79
F1-score: 0.00
Accuracy: 0.79
F1-score: 0.00
Accuracy: 0.78
F1-score: 0.26
Accuracy: 0.79
F1-score: 0.00


[0.7900701034856963, 0.05151634501772352]

In [10]:
KNN_1(['MISCED','FISCED','HISCED','PARED','MISCED_D','FISCED_D','HISCED_D','PAREDINT','BMMJ1','BFMJ2','HISEI'])

Accuracy: 0.72
F1-score: 0.23
Accuracy: 0.73
F1-score: 0.26
Accuracy: 0.72
F1-score: 0.25
Accuracy: 0.73
F1-score: 0.27
Accuracy: 0.73
F1-score: 0.25


[0.7259817453243966, 0.25320543716561617]

In [11]:
KNN_1(['ISCEDL'])

Accuracy: 0.50
F1-score: 0.42
Accuracy: 0.50
F1-score: 0.42
Accuracy: 0.49
F1-score: 0.42
Accuracy: 0.50
F1-score: 0.42
Accuracy: 0.51
F1-score: 0.43


[0.49670430421686157, 0.4206823529107238]

In [12]:
KNN_1(['OCOD1','OCOD2','OCOD3','PROGN'])

Accuracy: 0.74
F1-score: 0.33
Accuracy: 0.74
F1-score: 0.33
Accuracy: 0.74
F1-score: 0.33
Accuracy: 0.74
F1-score: 0.32
Accuracy: 0.73
F1-score: 0.30


[0.7382633268154125, 0.32363892640822306]

In [13]:
KNN_1(['COBN_S' ,'COBN_M' ,'COBN_F','PROGN'])

Accuracy: 0.79
F1-score: 0.20
Accuracy: 0.77
F1-score: 0.29
Accuracy: 0.79
F1-score: 0.15
Accuracy: 0.79
F1-score: 0.27
Accuracy: 0.81
F1-score: 0.29


[0.7917772675492701, 0.23924439960452007]

In [15]:
KNN_1(['COBN_S' ,'COBN_M' ,'COBN_F','PROGN','OCOD1','OCOD2','OCOD3'])

Accuracy: 0.74
F1-score: 0.35
Accuracy: 0.74
F1-score: 0.34
Accuracy: 0.74
F1-score: 0.35
Accuracy: 0.74
F1-score: 0.34
Accuracy: 0.74
F1-score: 0.34


[0.7407767090710966, 0.34260413905649445]

In [16]:
KNN_1(['SCCHANGE'])

Accuracy: 0.79
F1-score: 0.00
Accuracy: 0.79
F1-score: 0.00
Accuracy: 0.79
F1-score: 0.00
Accuracy: 0.79
F1-score: 0.00
Accuracy: 0.79
F1-score: 0.00


[0.7934607377476999, 0.0]

In [18]:
KNN_1(['PROGN','COBN_S' ,'COBN_M' ,'COBN_F','OCOD1','OCOD2','OCOD3','ISCEDL'])

Accuracy: 0.77
F1-score: 0.41
Accuracy: 0.76
F1-score: 0.41
Accuracy: 0.77
F1-score: 0.42
Accuracy: 0.77
F1-score: 0.42
Accuracy: 0.77
F1-score: 0.41


[0.7663362819969065, 0.4124269984193346]

In [35]:
KNN_1(['ST005Q01TA',
 'ST007Q01TA',
 'ST011D18TA',
 'ST011D19TA',
 'ST013Q01TA',
 'IC150Q06HA',
 'IC150Q09HA',
 'IC151Q06HA',
 'IC151Q08HA',
 'IC151Q09HA',
 'IC152Q06HA',
 'IC152Q08HA',
 'IC152Q09HA',
 'EC150Q06WA',
 'EC151Q03WA',
 'EC152Q01HA',
 'EC153Q02HA',
 'EC159Q01HA',
 'FL164Q01HA',
 'OCOD1',
 'OCOD2',
 'OCOD3',
 'PROGN',
 'ISCEDL',
 'BMMJ1',
 'BFMJ2',
 'HISEI',
 'BSMJ',
 'CHANGE',
 'ESCS',
 'HOMEPOS',
 'INFOCAR'])

Accuracy: 0.80
F1-score: 0.47
Accuracy: 0.80
F1-score: 0.49
Accuracy: 0.80
F1-score: 0.47
Accuracy: 0.80
F1-score: 0.48
Accuracy: 0.80
F1-score: 0.47


[0.7982502291390695, 0.4748646617150533]

- 基于SMOTE采样的KNN

In [28]:
def KNN_2(chosen_list) :
    #chosen_list是自变量特征
    chosen_list.append("REPEAT")
    df=data[chosen_list]   #取出所对应的列
    df=df.fillna(method='ffill')   
    df=df.fillna(method='bfill')   #数据预处理采用前后值填充方法
    kf = StratifiedKFold(n_splits = 5,random_state=2023,shuffle=True)    #分层kfold
    scaler = StandardScaler() #数据标准化
    X=np.array(df[chosen_list[:-1]]) #X为自变量特征
    y=np.array(df["REPEAT"])   #y是因变量特征（需要预测的特征）
    smo=SMOTE(sampling_strategy=1,random_state=42) #SMOTE采样
    f1_score_aver=0.0
    acc_aver=0.0
    for train,valid in kf.split(X,y) :
        #split(X,y)将训练集取4份，预测集取1份，train,vaild返回索引两者索引，轮换取
        X_train=np.array(X)[train]   #X_train是训练集自变量特征对应的数据
        y_train=np.array(y)[train]   #y_train是训练集因变量特征对应的数据
        X_test =np.array(X)[valid]   #X_test是预测集自变量特征对应的数据
        y_test=np.array(y)[valid]    #y_test是预测集因变量特征对应的数据
        scaler.fit(X_train)   
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        #根据X_train将X_test与X_train归一化处理，可以收敛更快
        X_resampled_smote,y_ressampled_smote=smo.fit_resample(X_train,y_train)
        clf = KNeighborsClassifier(n_neighbors=2)   
        clf.fit(X_resampled_smote, np.array(y_ressampled_smote).ravel())  #利用过采样后的训练集训练KNN分类器
        y_pred = clf.predict(X_test)   #将预测集的自变量特征对应的数据X_test输入分类器预测方法得到y_pred预测
        print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))   #acc
        print('F1-score: %.2f' % f1_score(y_test, y_pred))   #f1_score
        acc_aver+=accuracy_score(y_test, y_pred)
        f1_score_aver+=f1_score(y_test, y_pred)
    return [acc_aver/5,f1_score_aver/5]

In [34]:
KNN_2(['ST005Q01TA',
 'ST007Q01TA',
 'ST011D18TA',
 'ST011D19TA',
 'ST013Q01TA',
 'IC150Q06HA',
 'IC150Q09HA',
 'IC151Q06HA',
 'IC151Q08HA',
 'IC151Q09HA',
 'IC152Q06HA',
 'IC152Q08HA',
 'IC152Q09HA',
 'EC150Q06WA',
 'EC151Q03WA',
 'EC152Q01HA',
 'EC153Q02HA',
 'EC159Q01HA',
 'FL164Q01HA',
 'OCOD1',
 'OCOD2',
 'OCOD3',
 'PROGN',
 'ISCEDL',
 'BMMJ1',
 'BFMJ2',
 'HISEI',
 'BSMJ',
 'CHANGE',
 'ESCS',
 'HOMEPOS',
 'INFOCAR'])

Accuracy: 0.79
F1-score: 0.51
Accuracy: 0.79
F1-score: 0.52
Accuracy: 0.79
F1-score: 0.51
Accuracy: 0.79
F1-score: 0.51
Accuracy: 0.79
F1-score: 0.50


[0.7924886147183592, 0.5102747270699347]

- 基于ADASYN采样的KNN

In [31]:
def KNN_3(chosen_list) :
    #chosen_list是自变量特征
    chosen_list.append("REPEAT")
    df=data[chosen_list]   #取出所对应的列
    df=df.fillna(method='ffill')   
    df=df.fillna(method='bfill')   #数据预处理采用前后值填充方法
    kf = StratifiedKFold(n_splits = 5,random_state=2023,shuffle=True)    #分层kfold
    scaler = StandardScaler() #数据标准化
    X=np.array(df[chosen_list[:-1]]) #X为自变量特征
    y=np.array(df["REPEAT"])   #y是因变量特征（需要预测的特征）
    ada=ADASYN(sampling_strategy=1)
    f1_score_aver=0.0
    acc_aver=0.0
    for train,valid in kf.split(X,y) :
        #split(X,y)将训练集取4份，预测集取1份，train,vaild返回索引两者索引，轮换取
        X_train=np.array(X)[train]   #X_train是训练集自变量特征对应的数据
        y_train=np.array(y)[train]   #y_train是训练集因变量特征对应的数据
        X_test =np.array(X)[valid]   #X_test是预测集自变量特征对应的数据
        y_test=np.array(y)[valid]    #y_test是预测集因变量特征对应的数据
        scaler.fit(X_train)   
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        #根据X_train将X_test与X_train归一化处理，可以收敛更快
        #X_train_over, y_train_over = oversample.fit_resample(X_train, y_train)   #由于数据集分布不均匀，所以采用过采样方式
        X_resampled_ada,y_ressampled_ada=ada.fit_resample(X_train,y_train)
        clf = KNeighborsClassifier(n_neighbors=2)   
        clf.fit(X_resampled_ada, np.array(y_ressampled_ada).ravel())  #利用过采样后的训练集训练KNN分类器
        y_pred = clf.predict(X_test)   #将预测集的自变量特征对应的数据X_test输入分类器预测方法得到y_pred预测
        print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))   #acc
        print('F1-score: %.2f' % f1_score(y_test, y_pred))   #f1_score
        acc_aver+=accuracy_score(y_test, y_pred)
        f1_score_aver+=f1_score(y_test, y_pred)
    return [acc_aver/5,f1_score_aver/5] 

In [32]:
KNN_3(['PROGN','COBN_S' ,'COBN_M' ,'COBN_F','OCOD1','OCOD2','OCOD3','ISCEDL'])

Accuracy: 0.77
F1-score: 0.40
Accuracy: 0.76
F1-score: 0.40
Accuracy: 0.77
F1-score: 0.41
Accuracy: 0.77
F1-score: 0.42
Accuracy: 0.77
F1-score: 0.40


[0.768233154378601, 0.4060438847767175]

In [33]:
KNN_3(['ST005Q01TA',
 'ST007Q01TA',
 'ST011D18TA',
 'ST011D19TA',
 'ST013Q01TA',
 'IC150Q06HA',
 'IC150Q09HA',
 'IC151Q06HA',
 'IC151Q08HA',
 'IC151Q09HA',
 'IC152Q06HA',
 'IC152Q08HA',
 'IC152Q09HA',
 'EC150Q06WA',
 'EC151Q03WA',
 'EC152Q01HA',
 'EC153Q02HA',
 'EC159Q01HA',
 'FL164Q01HA',
 'OCOD1',
 'OCOD2',
 'OCOD3',
 'PROGN',
 'ISCEDL',
 'BMMJ1',
 'BFMJ2',
 'HISEI',
 'BSMJ',
 'CHANGE',
 'ESCS',
 'HOMEPOS',
 'INFOCAR'])

Accuracy: 0.78
F1-score: 0.50
Accuracy: 0.78
F1-score: 0.51
Accuracy: 0.78
F1-score: 0.50
Accuracy: 0.78
F1-score: 0.50
Accuracy: 0.79
F1-score: 0.50


[0.7829570871217518, 0.5027396581936558]

## 二分类神经网络模型预测

In [40]:
def neur_(chosen_list,n,m,max_iter_,flag) :
    chosen_list.append("REPEAT")
    df=data[chosen_list]
    df=df.fillna(method='ffill')
    df=df.fillna(method='bfill')
    kf = StratifiedKFold(n_splits = 5,random_state=2023,shuffle=True)
    X=np.array(df[chosen_list[:-1]])
    y=np.array(df["REPEAT"])
    scaler = StandardScaler()
    oversample = RandomOverSampler(sampling_strategy=1) #过采样
    ave_f1_score=0
    k=0
    acc_aver=0
    for train,valid in kf.split(X,y) :
        if flag :
            if k :
                break
            k+=1
        X_train=np.array(X)[train]
        y_train=np.array(y)[train]
        X_test =np.array(X)[valid]
        y_test=np.array(y)[valid]
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        X_train_over, y_train_over = oversample.fit_resample(X_train, y_train) #在X，y中进行过采样
        mlp=MLPClassifier(solver='adam', alpha=0, hidden_layer_sizes=(m, n), random_state=1,max_iter=max_iter_) #构造神经网络
        mlp.fit(X_train_over,y_train_over) #利用训练集进行神经网络训练
        y_pred = mlp.predict(X_test) #利用预测集进行预测
        ave_f1_score+=f1_score(y_test, y_pred)
        acc_aver+=accuracy_score(y_test, y_pred)
        print('Accuracy: %f' % accuracy_score(y_test, y_pred))
        print('F1-score: %f' % f1_score(y_test, y_pred))
    return [acc_aver/5,ave_f1_score/5]
    #return ave_f1_score

In [4]:
#在所有特征中找到单个训练时f1-score比较大的特征，经过实验发现这样所有此类特征结合起来训练可以使得f1-score更大
#暴力寻找
cols=data.columns
great_col=[]
i=0
for col in cols[7:400] :
    i+=1
    s=neur_([col],25,40,200,1)
    print(i,s)
    if s>0.37 :
        great_col.append(col)

1 0.3424697907456528
2 0.3424697907456528
3 0.3424697907456528
4 0.3424697907456528
5 0.2685700459714493
6 0.8846468502863375
7 0.30851241046143596
8 0.340778022438922
9 0.3365303718660609
10 0.38844232022593955
11 0.3424216401889223
12 0.367074210975263
13 0.3424697907456528
14 0.3371541044344101
15 0.3735772357723577
16 0.3440743944636679
17 0.35816814764183186
18 0.3254461335095836
19 0.3424697907456528
20 0.3424697907456528
21 0.3424697907456528
22 0.3424697907456528
23 0.24644243208279432
24 0.33086345077876905
25 0.3424697907456528
26 0.35494386091995656
27 0.35368956743002544
28 0.33663751214771626
29 0.2544052863436123
30 0.3329986613119143
31 0.09297625063163213
32 0.31319270647121916
33 0.3698612702049128
34 0.3705306018577427
35 0.3701158202876416
36 0.2987633965375103
37 0.34271968303381917
38 0.32863487159683336
39 0.21611129084438693
40 0.3375543375543375
41 0.3202357563850688
42 0.3623963626638138
43 0.3416309012875537
44 0.38323840520748575
45 0.20008661758336946
46 0.2

358 0.35472606863335343
359 0.347915949018257
360 0.3572978838849702
361 0.35309809627743416
362 0.3531442663378545
363 0.3491843110031239
364 0.3576006152268649
365 0.352148891881964
366 0.28967032967032963
367 0.27586206896551724
368 0.40035016049022465
369 0.38226037195994284
370 0.3940383830134749
371 0.8846468502863375
372 0.3012345679012346
373 0.37019108280254776
374 0.361428773118735
375 0.2814868278599783
376 0.36968856386332116
377 0.14435815440840566
378 0.420076377523186
379 0.342041312272175
380 0.35496465126094756
381 0.35716741148160874
382 0.3551046290424857
383 0.365280863218026
384 0.3583609060829728
385 0.361400512382579
386 0.35803051753972
387 0.35377811959238603
388 0.36944881889763775
389 0.39190981432360744
390 0.37936216396727335
391 0.40617151058485834
392 0.3246073298429319
393 0.32536479354237813


In [5]:
for col in cols[406:] :
    s=neur_([col],25,40,200,1)
    i+=1
    print(i,s)
    if s>0.38 :
        great_col.append(col)

394 0.40785645004849663
395 0.296680009418413
396 0.2942372881355932
397 0.3530872959545777
398 0.32752214918548156
399 0.3409041747782825
400 0.37286324786324787
401 0.40728884511204133
402 0.2666036398014654
403 0.4144547489606652
404 0.3393752410335519
405 0.3666282705282212
406 0.36168628035155664
407 0.27693677649154047
408 0.3211108700368843
409 0.3820532319391635
410 0.3714072970960536
411 0.3315987798313296
412 0.35144980329302056
413 0.3440555029959003
414 0.3294450335807706
415 0.3241795043536504
416 0.29206963249516443
417 0.3391632852945292
418 0.32406334492081884
419 0.2743558282208589
420 0.2939541348158443
421 0.3254390204607701
422 0.36419753086419754
423 0.3418270064135899
424 0.3497558810474922
425 0.34135060129509714
426 0.2911917098445596
427 0.31675675675675674
428 0.32333129210042866
429 0.27737881508078993
430 0.35515750463248913
431 0.2987537891545975
432 0.31312356101304684
433 0.2683131017161992
434 0.33392910336610065
435 0.36116264687693256
436 0.33176838810

In [8]:
great_col

['ST001D01T',
 'ST005Q01TA',
 'ST007Q01TA',
 'ST011D18TA',
 'ST011D19TA',
 'ST013Q01TA',
 'ST127Q01TA',
 'ST127Q02TA',
 'IC150Q06HA',
 'IC150Q09HA',
 'IC151Q06HA',
 'IC151Q08HA',
 'IC151Q09HA',
 'IC152Q06HA',
 'IC152Q08HA',
 'IC152Q09HA',
 'EC150Q06WA',
 'EC151Q03WA',
 'EC152Q01HA',
 'EC153Q02HA',
 'EC159Q01HA',
 'FL164Q01HA',
 'OCOD1',
 'OCOD2',
 'OCOD3',
 'GRADE',
 'PROGN',
 'ISCEDL',
 'BMMJ1',
 'BFMJ2',
 'HISEI',
 'BSMJ',
 'CHANGE',
 'ESCS',
 'HOMEPOS',
 'INFOCAR']

In [43]:
neur_(['ST005Q01TA',
 'ST007Q01TA',
 'ST011D18TA',
 'ST011D19TA',
 'ST013Q01TA',
 'IC150Q06HA',
 'IC150Q09HA',
 'IC151Q06HA',
 'IC151Q08HA',
 'IC151Q09HA',
 'IC152Q06HA',
 'IC152Q08HA',
 'IC152Q09HA',
 'EC150Q06WA',
 'EC151Q03WA',
 'EC152Q01HA',
 'EC153Q02HA',
 'EC159Q01HA',
 'FL164Q01HA',
 'OCOD1',
 'OCOD2',
 'OCOD3',
 'PROGN',
 'ISCEDL',
 'BMMJ1',
 'BFMJ2',
 'HISEI',
 'BSMJ',
 'CHANGE',
 'ESCS',
 'HOMEPOS',
 'INFOCAR'],30,40,500,0)

Accuracy: 0.782835
F1-score: 0.583258
Accuracy: 0.793480
F1-score: 0.596946
Accuracy: 0.784944
F1-score: 0.578336
Accuracy: 0.797155
F1-score: 0.602370
Accuracy: 0.805216
F1-score: 0.584996


[0.7927259552942015, 0.5891811346899444]

In [45]:
neur_(['ST005Q01TA',
 'ST007Q01TA',
 'ST011D18TA',
 'ST011D19TA',
 'ST013Q01TA',
 'IC150Q06HA',
 'IC150Q09HA',
 'IC151Q06HA',
 'IC151Q08HA',
 'IC151Q09HA',
 'IC152Q06HA',
 'IC152Q08HA',
 'IC152Q09HA',
 'EC150Q06WA',
 'EC151Q03WA',
 'EC152Q01HA',
 'EC153Q02HA',
 'EC159Q01HA',
 'FL164Q01HA',
 'OCOD1',
 'OCOD2',
 'OCOD3',
 'PROGN',
 'ISCEDL',
 'BMMJ1',
 'BFMJ2',
 'HISEI',
 'BSMJ',
 'CHANGE',
 'ESCS',
 'HOMEPOS',
 'INFOCAR'],30,10,500,0)

Accuracy: 0.801209
F1-score: 0.618082
Accuracy: 0.805335
F1-score: 0.622355
Accuracy: 0.790279
F1-score: 0.609578
Accuracy: 0.803794
F1-score: 0.633282
Accuracy: 0.791346
F1-score: 0.616558


[0.7983923838979529, 0.619971074340665]

In [46]:
neur_(['ST005Q01TA',
 'ST007Q01TA',
 'ST011D18TA',
 'ST011D19TA',
 'ST013Q01TA',
 'IC150Q06HA',
 'IC150Q09HA',
 'IC151Q06HA',
 'IC151Q08HA',
 'IC151Q09HA',
 'IC152Q06HA',
 'IC152Q08HA',
 'IC152Q09HA',
 'EC150Q06WA',
 'EC151Q03WA',
 'EC152Q01HA',
 'EC153Q02HA',
 'EC159Q01HA',
 'FL164Q01HA',
 'OCOD1',
 'OCOD2',
 'OCOD3',
 'PROGN',
 'ISCEDL',
 'BMMJ1',
 'BFMJ2',
 'HISEI',
 'BSMJ',
 'CHANGE',
 'ESCS',
 'HOMEPOS',
 'INFOCAR'],40,10,500,0)

Accuracy: 0.789118
F1-score: 0.608581
Accuracy: 0.813397
F1-score: 0.635141
Accuracy: 0.786603
F1-score: 0.606643
Accuracy: 0.808062
F1-score: 0.634949
Accuracy: 0.788975
F1-score: 0.611184


[0.7972308448591479, 0.6192997610250399]

In [49]:
neur_(['ST005Q01TA',
 'ST007Q01TA',
 'ST011D18TA',
 'ST011D19TA',
 'ST013Q01TA',
 'IC150Q06HA',
 'IC150Q09HA',
 'IC151Q06HA',
 'IC151Q08HA',
 'IC151Q09HA',
 'IC152Q06HA',
 'IC152Q08HA',
 'IC152Q09HA',
 'EC150Q06WA',
 'EC151Q03WA',
 'EC152Q01HA',
 'EC153Q02HA',
 'EC159Q01HA',
 'FL164Q01HA',
 'OCOD1',
 'OCOD2',
 'OCOD3',
 'PROGN',
 'ISCEDL',
 'BMMJ1',
 'BFMJ2',
 'HISEI',
 'BSMJ',
 'CHANGE',
 'ESCS',
 'HOMEPOS',
 'INFOCAR'],30,4,500,0)

Accuracy: 0.790422
F1-score: 0.613805
Accuracy: 0.804149
F1-score: 0.632399
Accuracy: 0.780202
F1-score: 0.604691
Accuracy: 0.797273
F1-score: 0.634459
Accuracy: 0.790990
F1-score: 0.621593


[0.7926072217664268, 0.6213893055134319]

In [61]:
neur_(['ST005Q01TA',
 'ST007Q01TA',
 'ST011D18TA',
 'ST011D19TA',
 'ST013Q01TA',
 'IC150Q06HA',
 'IC150Q09HA',
 'IC151Q06HA',
 'IC151Q08HA',
 'IC151Q09HA',
 'IC152Q06HA',
 'IC152Q08HA',
 'IC152Q09HA',
 'EC150Q06WA',
 'EC151Q03WA',
 'EC152Q01HA',
 'EC153Q02HA',
 'EC159Q01HA',
 'FL164Q01HA',
 'OCOD1',
 'OCOD2',
 'OCOD3',
 'PROGN',
 'ISCEDL',
 'BMMJ1',
 'BFMJ2',
 'HISEI',
 'BSMJ',
 'CHANGE',
 'ESCS',
 'HOMEPOS',
 'INFOCAR'],29,3,500,0)

Accuracy: 0.791015
F1-score: 0.619797
Accuracy: 0.796443
F1-score: 0.628113
Accuracy: 0.784114
F1-score: 0.608134
Accuracy: 0.797155
F1-score: 0.629413
Accuracy: 0.801186
F1-score: 0.626254


[0.7939824300012113, 0.622342343372086]